In [10]:
import sys
sys.path.insert(0, "..")
from ipdb import set_trace

In [11]:
from yolo.models import YOLOv1
from yolo.data.VOC_Dataset import VOC_Dataset
from utils.img_process import show_bbox
from torch.utils.data import DataLoader
from numpy import array
import torch

# output: 7 * 7 * 30
# S * S * ((x, y, w, h, confidence) * B=2 + C=20)
_voc_root = "/home/ubuntu/VOC2007/VOCdevkit/VOC2007/"
voc_ds = VOC_Dataset(_voc_root)

def collate_fn(data):
    # make img batch and label batch
    imgs, labels = zip(*data)

    # inhomegenous shape label, since each image has different number of objects
    # label (Batch size, # of objects in each image, 4 coords)
    return torch.tensor(array(imgs), dtype=torch.float), list(labels)

loader = DataLoader(voc_ds, batch_size=4, pin_memory=True, shuffle=True, num_workers=4, collate_fn=collate_fn)


In [12]:
import torch

test = torch.randn(3, 448, 448).unsqueeze(0).cuda()
yolo = YOLOv1().cuda()


lamba_coord = 5
lamba_noobj = 0.5


def IOU(coords, y_coords):
    """Calculate the intersection over union of two sets rectangles

    Keyword arguments
    coords -- output rect, with pct coordinates (x, y, w, h)
    y_coords -- label rect, with pct coordinates (xmin, ymin, xmax, ymax)"""
    xmin, ymin, xmax, ymax = (
        coords[0] - coords[2] / 2,
        coords[1] - coords[3] / 2,
        coords[0] + coords[2] / 2,
        coords[1] + coords[3] / 2,
    )

    x_inter = min(xmax, y_coords[2]) - max(xmin, y_coords[0])
    y_inter = min(ymax, y_coords[3]) - max(ymin, y_coords[1])

    if x_inter <= 0.0 or y_inter <= 0.0:
        return 0.0

    intersection = x_inter * y_inter

    overlapped_union = (xmax - xmin) * (ymax - ymin) + (y_coords[2] - y_coords[0]) * (
        y_coords[3] - y_coords[1]
    )
    
    return intersection / (overlapped_union - intersection)

coords = (0.25, 0.25, 0.5, 0.5)
y_coords1 = (0.25, 0.4, 0.75, 0.75)
y_coords2 = (0.4, 0.25, 0.75, 0.75)
y_coords3 = (0, 0, 0.5, 0.5)
y_coords4 = (0, 0, 0.5, 0)
y_coords5 = (0.5, 0, 0.75, 0.75)


print(IOU(coords, y_coords1))
print(IOU(coords, y_coords2))
print(IOU(coords, y_coords3))
print(IOU(coords, y_coords4))
print(IOU(coords, y_coords5))


0.062499999999999986
0.062499999999999986
1.0
0.0
0.0


In [15]:
for _id, sample in enumerate(loader):
    print(sample[0].shape)
    res = yolo(sample[0].cuda())
    print(res.shape)
    break
    

torch.Size([4, 3, 448, 448])
torch.Size([4, 30, 7, 7])
